In [1]:
import simpy
import numpy as np
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, timedelta

/var/folders/fm/scdzw77s0c51qj0q717pnn5w0000gn/T/ipykernel_2445/3731341879.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/fm/scdzw77s0c51qj0q717pnn5w0000gn/T/ipykernel_2445/3731341879.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## Objectives

In this notebook, we will be addressing subgroup B bonus question 2.

2. What is the potential impact of implementing an AI-driven real-time queue management system?
- Develop a prototype AI system for real-time queue and capacity adjustments.
- Set up a simulation environment to test and quantify the benefits of this system.

## Protoype AI system

We reuse attraction_names and estimated_capacity from subgroup B question 1 and 2 respectively.

### Key components and logic
1. Attraction Data
Each attraction is modeled as a dictionary containing details such as:
- name: Name of the attraction.
- estimated_capacity: Initial estimated capacity for guests.
- current_capacity: The dynamic capacity, which adjusts based on demand.
- optimal_capacity: Ideal capacity for the attraction.
- max_capacity & min_capacity: Upper and lower limits for capacity adjustments.
- queue: Number of guests waiting when demand exceeds capacity.
- utilization_log: A list tracking utilization over time for analysis.

2. Simulation Logic:
- Guest Demand: Simulated using a Poisson distribution around 80% of the estimated capacity.
- Capacity Adjustment:
  - If demand exceeds the current capacity, the attraction's capacity is increased (up to a max limit).
  - If demand is low and there’s no queue, the capacity is decreased (down to a minimum limit).
- Queue Management: If the demand exceeds capacity, guests are added to a queue.
- Utilization Metric: Calculated as the ratio of served guests to the current capacity. This is tracked over time for analysis.

In [2]:
# Estimated capacities per hour for each attraction
estimated_capacity = [400, 600, 500, 2000, 1200, 600, 500, 200, 1100, 400, 800, 400, 1000, 2000, 100, 100, 900, 400, 800, 250, 1000, 300]

# Names of attractions
attraction_names = [
    '[Hollywood] Trolls Hug Time Jubilee', '[New York] Sesame Street Spaghetti Chase', 
    '[New York] Lights, Camera, Action!', '[Sci-Fi City] Battlestar Galatica: HUMAN vs CYLON',
    '[Sci-Fi City] TRANSFORMERS The Ride: The Ultimate 3D Battle', '[Sci-Fi City] Accelerator', 
    '[Sci-Fi City] TRANSFORMERS: Voices of Cybertron', '[Sci-Fi City] Sci-Fi Games', 
    '[Ancient Egypt] Revenge of the Mummy', '[Ancient Egypt] Treasure Hunters', 
    '[The Lost World] Canopy Flyer', '[The Lost World] Dino-Soarin', 
    '[The Lost World] Jurassic Park Rapid Adventure', '[The Lost World] WaterWorld', 
    '[The Lost World] Hatched! Featuring Dr. Rooney', '[The Lost World] Raptor Encounter with Blue', 
    '[Far Far Away] Enchanted Airways', '[Far Far Away] Magic Potion Spin', 
    '[Far Far Away] Puss In Boots Giant Journey', '[Far Far Away] Donkey Live', 
    '[Far Far Away] Shrek 4-D Adventure', '[Far Far Away] Happily Ever After'
]

# Set up each attraction with a dictionary to track details
attractions = [
    {
        "name": attraction_names[i],
        "estimated_capacity": estimated_capacity[i],
        "current_capacity": estimated_capacity[i],
        "optimal_capacity": estimated_capacity[i],
        "max_capacity": estimated_capacity[i] * 1.5,  # Example max cap at 150% of estimated
        "min_capacity": max(estimated_capacity[i] * 0.5, 1),  # Min at 50%, not less than 1
        "queue": 0, 
        "utilization_log": []  
    }
    for i in range(len(attraction_names))
]

## Simulation Environment - Simpy
SimPy is used to model discrete event simulation. The model predicts utilisation per hour from 11am to 5pm (opening hours of USS).

In [22]:
# Attraction simulation logic
def simpy_attraction_simulation(env, attraction):
    last_logged_time = -1  # Track the last time logged

    while True:
        # Simulate the time of day (10AM to 9PM)
        current_hour = int(env.now // 60) + 10  # Convert from simulation time to actual hour
        current_minute = int(env.now % 60)
        current_time = current_hour * 60 + current_minute  # Total minutes from 10:00 AM
        
        # Log the data every hour (or the first time of the hour)
        if 10 <= current_hour <= 21 and current_minute == 0 and current_time != last_logged_time:
            # Adjust guest demand based on the current hour's estimated capacity
            guest_demand = np.random.poisson(attraction["estimated_capacity"] * 0.8)

            # Calculate the number of guests that can be served and adjust the queue
            if guest_demand > attraction["current_capacity"]:
                attraction["queue"] += guest_demand - attraction["current_capacity"]
                served_guests = attraction["current_capacity"]
            else:
                served_guests = guest_demand
                attraction["queue"] = max(0, attraction["queue"] - (attraction["current_capacity"] - guest_demand))

            # Adjust capacity based on demand and queue length
            if guest_demand > attraction["current_capacity"]:
                attraction["current_capacity"] = min(attraction["max_capacity"], attraction["current_capacity"] + 10)
            elif guest_demand < attraction["current_capacity"] * 0.7 and attraction["queue"] == 0:
                attraction["current_capacity"] = max(attraction["min_capacity"], attraction["current_capacity"] - 10)

            # Calculate and log utilization (ratio of served guests to current capacity)
            utilization = served_guests / attraction["current_capacity"]
            
            # Format time as HH:MM AM/PM
            time_str = (datetime(2000, 1, 1, current_hour, current_minute) + timedelta(minutes=env.now % 60)).strftime('%I:%M %p')

            # Log only if it's the first time for that hour
            attraction["utilization_log"].append((time_str, utilization))
            last_logged_time = current_time  # Update last logged time
        
        # Simulate a 10-minute interval
        yield env.timeout(10)

# Initialize the simulation environment
env = simpy.Environment()

# Add each attraction to the simulation environment
for attraction in attractions:
    env.process(simpy_attraction_simulation(env, attraction))

# Run the simulation from 10AM to 9PM
env.run(until=12 * 60)

# Create a list to store data for the DataFrame
data = []

# Extract the utilization log from each attraction
for attraction in attractions:
    for log in attraction["utilization_log"]:
        data.append([attraction["name"], log[0], log[1]])

# Create a DataFrame
simpy_predicted_utilisation = pd.DataFrame(data, columns=['Attraction Name', 'Time', 'Utilization'])

min_util = simpy_predicted_utilisation['Utilization'].min()
max_util = simpy_predicted_utilisation['Utilization'].max()

print("\nMax Utilisation:")
print(max_util)

print("\nMin Utilisation:")
print(min_util)



Max Utilisation:
1.0

Min Utilisation:
0.6
                          Attraction Name      Time  Utilization
0     [Hollywood] Trolls Hug Time Jubilee  10:00 AM     0.785000
1     [Hollywood] Trolls Hug Time Jubilee  11:00 AM     0.817500
2     [Hollywood] Trolls Hug Time Jubilee  12:00 PM     0.772500
3     [Hollywood] Trolls Hug Time Jubilee  01:00 PM     0.860000
4     [Hollywood] Trolls Hug Time Jubilee  02:00 PM     0.872500
...                                   ...       ...          ...
2371    [Far Far Away] Happily Ever After  05:00 PM     0.817241
2372    [Far Far Away] Happily Ever After  06:00 PM     0.841379
2373    [Far Far Away] Happily Ever After  07:00 PM     0.896552
2374    [Far Far Away] Happily Ever After  08:00 PM     0.765517
2375    [Far Far Away] Happily Ever After  09:00 PM     0.868966

[2376 rows x 3 columns]



In the simulated environment, utilization rates range from 0.6 to 0.99, which omits values below 0.6 that may indicate "under-utilization." This limited range risks skewing the AI system by not accounting for low-demand periods effectively. To address this, we normalize the utilization values, ensuring the full spectrum of demand levels is captured for accurate assessment and decision-making.

In [16]:
# Normalization step
min_util = simpy_predicted_utilisation['Utilization'].min()
max_util = simpy_predicted_utilisation['Utilization'].max()
simpy_predicted_utilisation['Normalized Utilization'] = (
    (simpy_predicted_utilisation['Utilization'] - min_util) / (max_util - min_util)
)

print(simpy_predicted_utilisation)

                          Attraction Name      Time  Utilization  \
0     [Hollywood] Trolls Hug Time Jubilee  10:00 AM     0.785000   
1     [Hollywood] Trolls Hug Time Jubilee  11:00 AM     0.817500   
2     [Hollywood] Trolls Hug Time Jubilee  12:00 PM     0.772500   
3     [Hollywood] Trolls Hug Time Jubilee  01:00 PM     0.860000   
4     [Hollywood] Trolls Hug Time Jubilee  02:00 PM     0.872500   
...                                   ...       ...          ...   
2107    [Far Far Away] Happily Ever After  05:00 PM     0.803448   
2108    [Far Far Away] Happily Ever After  06:00 PM     0.775862   
2109    [Far Far Away] Happily Ever After  07:00 PM     0.896552   
2110    [Far Far Away] Happily Ever After  08:00 PM     0.862069   
2111    [Far Far Away] Happily Ever After  09:00 PM     0.900000   

      Normalized Utilization  
0                   0.474359  
1                   0.557692  
2                   0.442308  
3                   0.666667  
4                   0.698718


## Visualisation - Dash and Plotly

We will be building a virtual map of USS showing attraction utilisation per hour from 10am to 9pm.

In [18]:
attractions_coordinates = {
    '[Hollywood] Trolls Hug Time Jubilee' : [1.2554, 103.8218],
    '[New York] Sesame Street Spaghetti Chase' : [1.2550, 103.8214],
    '[New York] Lights, Camera, Action!' : [1.2545, 103.8217], 
    '[Sci-Fi City] Battlestar Galatica: HUMAN vs CYLON' : [1.2531, 103.8218],
    '[Sci-Fi City] TRANSFORMERS The Ride: The Ultimate 3D Battle' : [1.2539, 103.8212], 
    '[Sci-Fi City] Accelerator' : [1.2537, 103.8217], 
    '[Sci-Fi City] TRANSFORMERS: Voices of Cybertron' : [1.2538, 103.8215], 
    '[Sci-Fi City] Sci-Fi Games' : [1.2535, 103.8220], 
    '[Ancient Egypt] Revenge of the Mummy' : [1.2531, 103.8232], 
    '[Ancient Egypt] Treasure Hunters' : [1.2539, 103.8231], 
    '[The Lost World] Canopy Flyer' : [1.2538, 103.8243], 
    '[The Lost World] Dino-Soarin' : [1.2536, 103.8244], 
    '[The Lost World] Jurassic Park Rapid Adventure' : [1.2533, 103.8239], 
    '[The Lost World] WaterWorld' : [1.2532, 103.8248], 
    '[The Lost World] Hatched! Featuring Dr. Rooney' : [1.2543, 103.8235], 
    '[The Lost World] Raptor Encounter with Blue' : [1.2543, 103.8238], 
    '[Far Far Away] Enchanted Airways' : [1.2549, 103.8234], 
    '[Far Far Away] Magic Potion Spin' : [1.2547, 103.8240], 
    '[Far Far Away] Puss In Boots Giant Journey' : [1.2544, 103.8247], 
    '[Far Far Away] Donkey Live' : [1.2546, 103.8245], 
    '[Far Far Away] Shrek 4-D Adventure' : [1.2550, 103.8239], 
    '[Far Far Away] Happily Ever After' : [1.2550, 103.8236]
}

simpy_dashboard = simpy_predicted_utilisation.copy()

simpy_dashboard['Latitude'] = simpy_dashboard['Attraction Name'].map(lambda x: attractions_coordinates.get(x, [None, None])[0])
simpy_dashboard['Longitude'] = simpy_dashboard['Attraction Name'].map(lambda x: attractions_coordinates.get(x, [None, None])[1])

print(simpy_dashboard)

                          Attraction Name      Time  Utilization  \
0     [Hollywood] Trolls Hug Time Jubilee  10:00 AM     0.785000   
1     [Hollywood] Trolls Hug Time Jubilee  11:00 AM     0.817500   
2     [Hollywood] Trolls Hug Time Jubilee  12:00 PM     0.772500   
3     [Hollywood] Trolls Hug Time Jubilee  01:00 PM     0.860000   
4     [Hollywood] Trolls Hug Time Jubilee  02:00 PM     0.872500   
...                                   ...       ...          ...   
2107    [Far Far Away] Happily Ever After  05:00 PM     0.803448   
2108    [Far Far Away] Happily Ever After  06:00 PM     0.775862   
2109    [Far Far Away] Happily Ever After  07:00 PM     0.896552   
2110    [Far Far Away] Happily Ever After  08:00 PM     0.862069   
2111    [Far Far Away] Happily Ever After  09:00 PM     0.900000   

      Normalized Utilization  Latitude  Longitude  
0                   0.474359    1.2554   103.8218  
1                   0.557692    1.2554   103.8218  
2                   0.44230

To run the dashboard, run the code below and copy http://127.0.0.1:8050/ to your browser.

In [23]:
# Create Dash App
app = dash.Dash(__name__)

# Round off Utilization to 2 decimal places
simpy_dashboard['Normalized Utilization'] = simpy_dashboard['Normalized Utilization'].round(2)

# Define Map with Plotly Express
map_figure = px.scatter_mapbox(
    simpy_dashboard, 
    lat="Latitude", 
    lon="Longitude", 
    size="Normalized Utilization", 
    color="Normalized Utilization",
    hover_name="Attraction Name", 
    hover_data={"Latitude": False, "Longitude": False, "Time": True, "Normalized Utilization": True},  # Disable Latitude and Longitude
    color_continuous_scale="Plasma_r", 
    size_max=20, 
    zoom=15
)

# Set Map style
map_figure.update_layout(mapbox_style="open-street-map", title="Attraction Utilization")

# Layout for the Dash app
app.layout = html.Div([
    html.H1("Universal Studios Singapore - Attraction Utilization"),
    
    # Slider for time
    dcc.Slider(
        id='time-slider',
        min=0,
        max=len(simpy_dashboard['Time'].unique()) - 1,
        step=1,
        marks={i: time for i, time in enumerate(simpy_dashboard['Time'].unique())},
        value=0  # Initial value (default to the first time)
    ),
    
    # Map
    dcc.Graph(
        figure=map_figure, 
        id="utilization-map", 
        style={"height": "800px", "width": "100%"}  # Set map size (height and width)
    ),
])

# Callback to update map based on time slider
@app.callback(
    Output('utilization-map', 'figure'),
    [Input('time-slider', 'value')]
)
def update_map(selected_time_index):
    selected_time = simpy_dashboard['Time'].unique()[selected_time_index]
    
    # Filter and remove duplicates by attraction name for the selected time
    filtered_simpy_dashboard = simpy_dashboard[simpy_dashboard['Time'] == selected_time].drop_duplicates(subset=['Attraction Name'])
    
    # Round off Utilization to 2 decimal places for filtered data
    filtered_simpy_dashboard["Normalized Utilization"] = filtered_simpy_dashboard["Normalized Utilization"].round(2)
    
    # Create the map figure with filtered data
    map_figure = px.scatter_mapbox(
        filtered_simpy_dashboard, 
        lat="Latitude", 
        lon="Longitude", 
        size="Normalized Utilization", 
        color="Normalized Utilization",
        hover_name="Attraction Name", 
        hover_data={"Latitude": False, "Longitude": False, "Time": True, "Normalized Utilization": True},
        color_continuous_scale="Plasma_r", 
        size_max=20, 
        zoom=16.5
    )
    map_figure.update_layout(mapbox_style="open-street-map", title=f"Attraction Utilization - {selected_time}")
    
    return map_figure

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
